# Imports

In [1]:
from urllib.parse import urlparse, parse_qs, urlencode
from requests import get, post, Session
from getpass import getpass
from re import findall

# Get token

## Urls def

In [2]:
beatport_base_url = 'https://api.beatport.com/v4'
beatport_redirect_uri = beatport_base_url + '/auth/o/post-message/'

## Get client id

In [3]:
client_id = ''
html = get('https://api.beatport.com/v4/docs/').content.decode('utf-8')
scripts_matches = findall(r"src=.(.*js)", html)
for script_url in scripts_matches:
    url = 'https://api.beatport.com{}'.format(script_url)
    js = get(url.format(script_url)).content.decode('utf-8')
    client_id_matches = findall(r"API_CLIENT_ID: \'(.*)\'", js)
    if client_id_matches:
        client_id = client_id_matches[0]
client_id

'0GIvkCltVIuPkkwSJHp6NDb3s0potTjLBQr388Dd'

## Login

In [4]:
sess = Session()

In [5]:
response = sess.post(url=beatport_base_url + '/auth/login/', 
                     json={'username': input("Enter your username: "),
                           'password': getpass("Enter your password: ")})
response.json()

{'username': 'sonwever',
 'first_name': 'Mathis',
 'last_name': 'NIVEAU',
 'email': 'mathis.terrariose@gmail.com',
 'email_confirmed': False,
 'register_ip_address_ntoa': '134.214.58.26',
 '_next': 'https://www.beatport.com'}

## Authorization code

In [6]:
query = urlencode({
    'client_id': client_id,
    'response_type': 'code',
    'redirect_uri': beatport_redirect_uri
})
response = sess.get(url=f'{beatport_base_url}/auth/o/authorize/?{query}', allow_redirects=False)
response.url

'https://api.beatport.com/v4/auth/o/authorize/?client_id=0GIvkCltVIuPkkwSJHp6NDb3s0potTjLBQr388Dd&response_type=code&redirect_uri=https%3A%2F%2Fapi.beatport.com%2Fv4%2Fauth%2Fo%2Fpost-message%2F'

In [7]:
next_url = urlparse(beatport_base_url + response.headers['Location'])
auth_code = parse_qs(next_url.query)['code'][0]
auth_code

'p8wLCPeqy94eyS0sTXMQSJrRgUabXx'

In [8]:
query = urlencode({
    'code': auth_code,
    'grant_type': 'authorization_code',
    'redirect_uri': beatport_redirect_uri,
    'client_id': client_id
})
response = sess.post(url=beatport_base_url+'/auth/o/token/?'+query)
token = response.json()
token


{'access_token': 'orq7zvrI1xKoJtyvdtTiKaZpvpItP0',
 'expires_in': 36000,
 'token_type': 'Bearer',
 'scope': 'app:docs user:dj',
 'refresh_token': 'qMYP5QdrEwf0gwpfZJMT5tuc2iLtk5'}

## Test requests

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0',
           'Authorization': f'Bearer {token["access_token"]}'}

In [10]:
response = get(url=beatport_base_url+'/catalog/search/', 
               headers=headers, 
               params={"q": "lay low tiesto", 'type': 'tracks', 'per_page': '5'})
response.json()

{'tracks': [{'artists': [{'id': 4178,
     'image': {'id': 26803617,
      'uri': 'https://geo-media.beatport.com/image_size/590x404/846683c6-4b4d-44db-8ebe-13d5d3cd5f92.jpg',
      'dynamic_uri': 'https://geo-media.beatport.com/image_size/{w}x{h}/846683c6-4b4d-44db-8ebe-13d5d3cd5f92.jpg'},
     'name': 'Tiesto',
     'slug': 'tiesto',
     'url': 'https://api.beatport.com/v4/catalog/artists/4178/'}],
   'publish_status': 'published',
   'available_worldwide': False,
   'bpm': 122,
   'catalog_number': '5054197482496',
   'current_status': {'id': 10,
    'name': 'General Content',
    'url': 'https://api.beatport.com/v4/auxiliary/current-status/10/'},
   'encoded_date': '2022-12-02T13:08:56-07:00',
   'exclusive': False,
   'free_downloads': [],
   'free_download_start_date': None,
   'free_download_end_date': None,
   'genre': {'id': 39,
    'name': 'Dance / Electro Pop',
    'slug': 'dance-electro-pop',
    'url': 'https://api.beatport.com/v4/catalog/genres/39/'},
   'id': 17189283,
